In [2]:
import os
import wget
import pandas as pd
from collections import defaultdict

In [3]:
if not os.path.exists("DiscreteFactors.py"):
    !wget 'https://raw.githubusercontent.com/UNSW-COMP9418/libraries/main/DiscreteFactors.py'
if not os.path.exists("Graph.py"):
    !wget 'https://raw.githubusercontent.com/UNSW-COMP9418/libraries/main/Graph.py'
if not os.path.exists("BayesNet_VE.py"):
    !wget 'https://raw.githubusercontent.com/UNSW-COMP9418/libraries/main/BayesNet_VE.py'    


    

In [4]:
from DiscreteFactors import Factor
from Graph import Graph
from BayesNet_VE import BayesNet
from HiddenMarkovModel import HiddenMarkovModel

In [5]:
training = pd.read_csv('data1.csv')


In [6]:
training.head

<bound method NDFrame.head of       Unnamed: 0 motion_sensor1 motion_sensor2 motion_sensor3 motion_sensor4  \
0              0      no motion      no motion      no motion      no motion   
1              1      no motion         motion      no motion      no motion   
2              2      no motion      no motion      no motion         motion   
3              3      no motion         motion      no motion      no motion   
4              4      no motion         motion      no motion      no motion   
...          ...            ...            ...            ...            ...   
2395        2395      no motion      no motion      no motion      no motion   
2396        2396      no motion      no motion      no motion         motion   
2397        2397      no motion         motion      no motion      no motion   
2398        2398      no motion      no motion      no motion         motion   
2399        2399      no motion      no motion      no motion      no motion   

     moti

In [7]:
training.columns

Index(['Unnamed: 0', 'motion_sensor1', 'motion_sensor2', 'motion_sensor3',
       'motion_sensor4', 'motion_sensor5', 'motion_sensor6', 'camera1',
       'camera2', 'camera3', 'camera4', 'robot1', 'robot2', 'door_sensor1',
       'door_sensor2', 'door_sensor3', 'door_sensor4', 'time', 'r1', 'r2',
       'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13',
       'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23',
       'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33',
       'r34', 'c1', 'c2', 'outside'],
      dtype='object')

In [8]:
training_small = training[['r1', 'door_sensor1', 'motion_sensor1']]

In [9]:
training = training.drop('Unnamed: 0',axis = 1)
training.columns

Index(['motion_sensor1', 'motion_sensor2', 'motion_sensor3', 'motion_sensor4',
       'motion_sensor5', 'motion_sensor6', 'camera1', 'camera2', 'camera3',
       'camera4', 'robot1', 'robot2', 'door_sensor1', 'door_sensor2',
       'door_sensor3', 'door_sensor4', 'time', 'r1', 'r2', 'r3', 'r4', 'r5',
       'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16',
       'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26',
       'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'c1', 'c2',
       'outside'],
      dtype='object')

In [10]:
def learn_outcome_space(data):
    '''
    Arguments:
        data:    A pandas dataframe
    Returns: 
        Nothing. Store the learned outcome space in self.outcomeSpace
        
    '''
    outcomeSpace = {}
    for column in list(data.columns):
        values = list(data[column].unique())
        outcomeSpace[column] = tuple(values)

    return outcomeSpace

In [11]:
outcomeSpace = learn_outcome_space(training_small)

In [12]:
outcomeSpace

{'r1': (0, 1, 2, 3, 7, 4, 5),
 'door_sensor1': (0, 1, 2, 4, 3),
 'motion_sensor1': ('no motion', 'motion')}

In [13]:
training_small['r1']>0

0       False
1       False
2       False
3       False
4       False
        ...  
2395    False
2396    False
2397    False
2398    False
2399    False
Name: r1, Length: 2400, dtype: bool

In [14]:
training_small[training_small['r1']>0]

,r1,door_sensor1,motion_sensor1
62,1,0,motion
63,1,0,motion
64,1,0,motion
65,1,0,motion
66,1,0,motion
...,...,...,...
2236,1,0,motion
2237,1,0,motion
2238,1,0,motion
2239,1,0,motion


In [15]:
training_small.loc[training_small['r1'] > 0, 'r1'] = 'on'
training_small.loc[training_small['r1'] == 0, 'r1'] = 'off'

/var/folders/j9/frkgv22j2m712ncxzqgzsn900000gn/T/ipykernel_24236/3478238970.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'on' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  training_small.loc[training_small['r1'] > 0, 'r1'] = 'on'


In [16]:
training_small.loc[training_small['door_sensor1'] > 0, 'door_sensor1'] = 'door'
training_small.loc[training_small['door_sensor1'] == 0, 'door_sensor1'] = 'no door'

/var/folders/j9/frkgv22j2m712ncxzqgzsn900000gn/T/ipykernel_24236/3172802551.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'door' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  training_small.loc[training_small['door_sensor1'] > 0, 'door_sensor1'] = 'door'


In [17]:
learn_outcome_space(training_small)

{'r1': ('off', 'on'),
 'door_sensor1': ('no door', 'door'),
 'motion_sensor1': ('no motion', 'motion')}

In [18]:
(training_small['r1']=='off').sum()

679

In [19]:
(training_small['r1']=='on').sum()

1721

In [20]:
1721+679

2400

In [21]:
1721/2400

0.7170833333333333

In [22]:
len(training_small['r1'])

2400

In [23]:
transition_counts = defaultdict(int)

for i in range (0, len(training_small['r1'])-1):
    transition_counts[training_small['r1'][i],training_small['r1'][i+1]]+=1



In [24]:
for key in transition_counts:
    transition_counts[key]= transition_counts[key]/2400


In [25]:
c = 0
for key in transition_counts:
    c+=transition_counts[key]

In [26]:
training_small

,r1,door_sensor1,motion_sensor1
0,off,no door,no motion
1,off,no door,no motion
2,off,no door,no motion
3,off,no door,no motion
4,off,no door,no motion
...,...,...,...
2395,off,no door,no motion
2396,off,no door,no motion
2397,off,no door,no motion
2398,off,no door,no motion


In [27]:

((training_small['r1']=='on') & (training_small['motion_sensor1']=='motion')).sum()/2400

0.6808333333333333

In [28]:
['on','off']*['no motion', 'motion']

TypeError: can't multiply sequence by non-int of type 'list'

In [ ]:
outcomeSpace = {
    'r1':('on','off'),
    'r1_next':('on','off'),
    'motion': ('no motion', 'motion'),
    'door': ('no door', 'door')
}
r1Emission = Factor(('r1', 'motion', 'door'), outcomeSpace)
 
# prob v1 given v2, v3
for v1 in outcomeSpace['r1']:
    for v2 in outcomeSpace['motion']:
        for v3 in outcomeSpace['door']:
            counts_v2v3 = ((training_small['motion_sensor1']==v2) & (training_small['door_sensor1']==v3)).sum()+2 #2 for smoothing
            counts_all = ((training_small['r1']==v1) & (training_small['motion_sensor1']==v2) & (training_small['door_sensor1']==v3)).sum()+1
            r1Emission[v1, v2,v3]= counts_all/counts_v2v3

In [ ]:
print(r1Emission)

╒══════╤═══════════╤═════════╤═══════════╕
│ r1   │ motion    │ door    │        Pr │
╞══════╪═══════════╪═════════╪═══════════╡
│ on   │ no motion │ no door │ 0.130363  │
├──────┼───────────┼─────────┼───────────┤
│ on   │ no motion │ door    │ 0.178571  │
├──────┼───────────┼─────────┼───────────┤
│ on   │ motion    │ no door │ 0.936548  │
├──────┼───────────┼─────────┼───────────┤
│ on   │ motion    │ door    │ 0.941176  │
├──────┼───────────┼─────────┼───────────┤
│ off  │ no motion │ no door │ 0.869637  │
├──────┼───────────┼─────────┼───────────┤
│ off  │ no motion │ door    │ 0.821429  │
├──────┼───────────┼─────────┼───────────┤
│ off  │ motion    │ no door │ 0.0634518 │
├──────┼───────────┼─────────┼───────────┤
│ off  │ motion    │ door    │ 0.0588235 │
╘══════╧═══════════╧═════════╧═══════════╛



In [29]:
training_small.shape[0]

2400

In [31]:
##################
# Test code

# possible outcomes, by variable
outcomeSpace = {
    'r1':('on','off'),
    'r1_next':('on','off'),
    'motion': ('no motion', 'motion'),
    'door': ('no door', 'door')
}

r1Start = Factor(('r1',), outcomeSpace)
r1Start['off'] = (training_small['r1']=='on').sum()/training_small.shape[0]
r1Start['on'] = (training_small['r1']=='off').sum()/training_small.shape[0]
r1Transition = Factor(('r1', 'r1_next'), outcomeSpace)
r1Transition['on', 'on'] = transition_counts[('on', 'on')]
r1Transition['on', 'off'] = transition_counts[('on', 'off')]
r1Transition['off', 'on'] = transition_counts[('off', 'on')]
r1Transition['off', 'off'] = transition_counts[('off', 'off')]
r1Emission = Factor(('r1', 'motion', 'door'), outcomeSpace)
 
# prob v1 given v2, v3
for v1 in outcomeSpace['r1']:
    for v2 in outcomeSpace['motion']:
        for v3 in outcomeSpace['door']:
            counts_v2v3 = ((training_small['motion_sensor1']==v2) & (training_small['door_sensor1']==v3)).sum()+2 #2 for smoothing (hardcoded)
            counts_all = ((training_small['r1']==v1) & (training_small['motion_sensor1']==v2) & (training_small['door_sensor1']==v3)).sum()+1
            r1Emission[v1, v2,v3]= counts_all/counts_v2v3

variable_remap = {
    'r1_next': 'r1'
}

hmm = HiddenMarkovModel(r1Start, r1Transition,r1Emission, variable_remap)
print(hmm.forward(motion='motion', door = 'no door'))

hmm = HiddenMarkovModel(r1Start, r1Transition,r1Emission, variable_remap)
print(hmm.forward())

╒══════╤═══════════╕
│ r1   │        Pr │
╞══════╪═══════════╡
│ on   │ 0.192713  │
├──────┼───────────┤
│ off  │ 0.0126702 │
╘══════╧═══════════╛

╒══════╤══════════╕
│ r1   │       Pr │
╞══════╪══════════╡
│ on   │ 0.449947 │
├──────┼──────────┤
│ off  │ 0.362091 │
╘══════╧══════════╛



In [40]:
import numpy as np

In [41]:
factor = hmm.forward(motion='motion', door = 'no door')

np.argmax([factor['on'], factor['off']])

0

In [43]:
factor.table.argmax()

0

In [90]:
hmm =  HiddenMarkovModel(r1Start, r1Transition,r1MotionEmission, variable_remap)
timeLine = hmm.forwardBatch(2, motion=['no motion', 'motion'])
for t in range(len(timeLine)):
    print("Time: ", t)
    print(timeLine[t])
    print()

Time:  0
╒══════╤═══════════╕
│ r1   │        Pr │
╞══════╪═══════════╡
│ on   │ 0.0185351 │
├──────┼───────────┤
│ off  │ 0.0197039 │
╘══════╧═══════════╛


Time:  1
╒══════╤═════════════╕
│ r1   │          Pr │
╞══════╪═════════════╡
│ on   │ 0.00905442  │
├──────┼─────────────┤
│ off  │ 0.000250135 │
╘══════╧═════════════╛




In [5]:
##################
# Test code

# possible outcomes, by variable
outcomeSpace = {
    'Weather':('sun','rain'),
    'Weather_next':('sun','rain'),
    'Umbrella': ('umbrella', 'no_umbrella'),
}

weatherStart = Factor(('Weather',), outcomeSpace)
weatherStart['sun'] = 0.5
weatherStart['rain'] = 0.5
weatherTransition = Factor(('Weather', 'Weather_next'), outcomeSpace)
weatherTransition['sun', 'sun'] = 0.7
weatherTransition['sun', 'rain'] = 0.3
weatherTransition['rain', 'sun'] = 0.3
weatherTransition['rain', 'rain'] = 0.7
weatherEmission = Factor(('Weather', 'Umbrella'), outcomeSpace)
weatherEmission['sun', 'umbrella'] = 0.2
weatherEmission['sun', 'no_umbrella'] = 0.8
weatherEmission['rain', 'umbrella'] = 0.9
weatherEmission['rain', 'no_umbrella'] = 0.1

variable_remap = {
    'Weather_next': 'Weather'
}

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
print(hmm.forward(Umbrella='umbrella'))

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
print(hmm.forward())

╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │ 0.1  │
├───────────┼──────┤
│ rain      │ 0.45 │
╘═══════════╧══════╛

╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │  0.5 │
├───────────┼──────┤
│ rain      │  0.5 │
╘═══════════╧══════╛



In [6]:
hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
timeLine = hmm.forwardBatch(2, Umbrella=['umbrella', 'umbrella'])
for t in range(len(timeLine)):
    print("Time: ", t)
    print(timeLine[t])
    print()

Time:  0
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │ 0.1  │
├───────────┼──────┤
│ rain      │ 0.45 │
╘═══════════╧══════╛


Time:  1
╒═══════════╤════════╕
│ Weather   │     Pr │
╞═══════════╪════════╡
│ sun       │ 0.041  │
├───────────┼────────┤
│ rain      │ 0.3105 │
╘═══════════╧════════╛


